# Introduction

QA plots for the INTT detector

In [1]:
# imports to write dynamic markdown contents
import os
from IPython.display import display, Markdown, Latex
from IPython.display import HTML

In [2]:
# turn off/on code for the result HTML page

display(Markdown('*For the result HTML page:* '))
    
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 $('div.jp-CodeMirrorEditor').hide(); // for newer versions of nbviewer.org
 } else {
 $('div.input').show();
 $('div.jp-CodeMirrorEditor').show(); // for newer versions of nbviewer.org
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

*For the result HTML page:* 

In [3]:
import os.path

# readme file of the macros, available if run under JenkinsCI
# https://github.com/sPHENIX-Collaboration/utilities/blob/master/jenkins/built-test/test-tracking-qa.sh 

macro_markdown = 'Fun4All-macros-README.md'

if os.path.isfile(macro_markdown) :
    with open(macro_markdown, 'r') as file:
        display(Markdown(file.read()))

## `pyROOT` env check

In [4]:
import ROOT

OFFLINE_MAIN = os.getenv("OFFLINE_MAIN")
if OFFLINE_MAIN is not None:
    display(Markdown(f"via sPHENIX software distribution at `{OFFLINE_MAIN}`"))

via sPHENIX software distribution at `/cvmfs/sphenix.sdcc.bnl.gov/alma9.2-gcc-14.2.0/release/release_ana/ana.483`

## Plotting source code

In [5]:
import subprocess

try:
    git_url = \
        subprocess.run(['git','remote','get-url','origin'], stdout=subprocess.PIPE)\
        .stdout.decode('utf-8').strip()\
        .replace('git@github.com:','https://github.com/')

    display(Markdown(f"View the source code repository at {git_url}"))
except: # catch *all* exceptions
    #     well do nothing
    pass

View the source code repository at https://github.com/sPHENIX-Collaboration/QA-gallery.git

## JenkinsCI information (if available)

In [6]:
display(Markdown('Some further details about the QA run, if executed under the Jenkins CI:'))

checkrun_repo_commit = os.getenv("checkrun_repo_commit")
if checkrun_repo_commit is not None:
    display(Markdown(f"* The commit being checked is {checkrun_repo_commit}"))
        
ghprbPullLink = os.getenv("ghprbPullLink")
if ghprbPullLink is not None:
    display(Markdown(f"* Link to the pull request: {ghprbPullLink}"))

BUILD_URL =  os.getenv("BUILD_URL")
if BUILD_URL is not None:
    display(Markdown(f"* Link to the build: {BUILD_URL}"))

git_url_macros =  os.getenv("git_url_macros")
sha_macros =  os.getenv("sha_macros")
if git_url_macros is not None:
    display(Markdown(f"* Git repo for macros: {git_url_macros} , which merges `{sha_macros}` and the QA tracking branch"))

RUN_ARTIFACTS_DISPLAY_URL = os.getenv("RUN_ARTIFACTS_DISPLAY_URL")
if RUN_ARTIFACTS_DISPLAY_URL is not None:
    display(Markdown(f"* Download the QA ROOT files: {RUN_ARTIFACTS_DISPLAY_URL}"))

JENKINS_URL  = os.getenv("JENKINS_URL")
if JENKINS_URL is not None:
    display(Markdown(f"Automatically generated by [sPHENIX Jenkins continuous integration]({JENKINS_URL}) [![sPHENIX](https://raw.githubusercontent.com/sPHENIX-Collaboration/utilities/master/jenkins/material/sphenix-logo-white-bg-72p.png)](https://www.sphenix.bnl.gov/web/) &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; [![jenkins.io](https://raw.githubusercontent.com/sPHENIX-Collaboration/utilities/master/jenkins/material/jenkins_logo_title-72p.png)](https://jenkins.io/)"))

Some further details about the QA run, if executed under the Jenkins CI:


# Initialization

In [7]:
%%cpp -d

#include "QA_Draw_Utility.C"

#include <sPhenixStyle.C>

#include <TFile.h>
#include <TLine.h>
#include <TString.h>
#include <TTree.h>
#include <cassert>
#include <cmath>

In [8]:
%%cpp

SetsPhenixStyle();
TVirtualFitter::SetDefaultFitter("Minuit2");

// test sPHENIX lib load
// gSystem->Load("libg4eval.so");

// test libs
// gSystem->ListLibraries();

sPhenixStyle: Applying nominal settings.
sPhenixStyle: ROOT6 mode


In [9]:
%jsroot on

## Inputs and file checks

In [10]:
%%cpp -d

void INTTRawHitChip() {
    // Open the ROOT file
    TFile* file = TFile::Open("prdf_reconstruction53877_qa.root");
    if (!file || file->IsZombie()) {
        std::cerr << "Error opening root file" << std::endl;
        return;
    }

    // Create a canvas
    TCanvas* canvas = new TCanvas("canvas", "INTT Plots 0", 1000, 800);
    canvas->Divide(4, 4);

    // Get and draw the histograms
    for (int lad = 0; lad < 14; lad++) {
        std::ostringstream hist_name;
        hist_name << "h_InttRawHitQA_intt" << 6 << "_" << lad;
        TH1* h_nhits = dynamic_cast<TH1*>(file->Get(hist_name.str().c_str()));
        canvas->cd(lad + 1);
        gPad->SetRightMargin(0.13);
        gPad->SetLeftMargin(0.13);
        if (h_nhits) {
            h_nhits->SetTitle(Form("FEE %d", lad));
            h_nhits->Draw("colz");
        } else {
            std::cerr << "Histogram " << hist_name.str() << " not found" << std::endl;
        }
    }

    // Add text to the last pad
    TText PrintRun;
    PrintRun.SetTextFont(62);
    PrintRun.SetTextSize(0.06);
    PrintRun.SetNDC();          // set to normalized coordinates
    PrintRun.SetTextAlign(23);  // center/top alignment
    std::string runstring1 = "";
    canvas->cd(15);
    PrintRun.DrawText(0.5, 0.5, runstring1.c_str());

    // Update and save the canvas
    canvas->Update();
    //canvas->SaveAs("INTT_Plots.pdf");

    std::cout << "Plotting completed" << std::endl;
}

## Utilities (invisible to HTML output)

In [11]:
%%cpp
INTTRawHitChip();

Plotting completed


# Azimuthal arc-length, $r \Delta\phi$, residuals, errors and pulls

## Residuals

In [12]:
%%cpp -d

void INTTRawHitSummaryDrawer_v2() {
    // Open the ROOT file
    TFile* file = TFile::Open("prdf_reconstruction53877_qa.root");
    if (!file || file->IsZombie()) {
        std::cerr << "Error opening root file" << std::endl;
        return;
    }

    gStyle->SetPalette(kRainBow);

    TCanvas* m_canvas = new TCanvas("m_canvas", "INTT Hits Summary", 800, 600);
    m_canvas->Divide(2, 4);

    // Get histograms
    TH1* nhit = dynamic_cast<TH1*>(file->Get("h_InttRawHitQA_nhit"));
    TH1* nhitnorth = dynamic_cast<TH1*>(file->Get("h_InttRawHitQA_nhit_north"));
    TH1* nhitsouth = dynamic_cast<TH1*>(file->Get("h_InttRawHitQA_nhit_south"));
    TH1* pid = dynamic_cast<TH1*>(file->Get("h_InttRawHitQA_pid"));
    TH1* adc = dynamic_cast<TH1*>(file->Get("h_InttRawHitQA_adc"));
    TH1* bco = dynamic_cast<TH1*>(file->Get("h_InttRawHitQA_bco"));
    TH1* bcofull = dynamic_cast<TH1*>(file->Get("h_InttRawHitQA_bco_full"));

    // Draw histograms to canvas
    if (nhit) {
        m_canvas->cd(1);
        nhit->GetXaxis()->SetTitle("N_{hit}");
        nhit->Draw();
    } else {
        std::cerr << "Histogram nhit not found" << std::endl;
    }

    if (nhitnorth) {
        m_canvas->cd(2);
        nhitnorth->GetYaxis()->SetTitle("N_{hit}");
        nhitnorth->SetTitle("INTT Hits North");
        nhitnorth->Draw();
    } else {
        std::cerr << "Histogram nhitnorth not found" << std::endl;
    }

    if (nhitsouth) {
        m_canvas->cd(3);
        nhitsouth->GetYaxis()->SetTitle("N_{hit}");
        nhitsouth->SetTitle("INTT Hits South");
        nhitsouth->Draw();
    } else {
        std::cerr << "Histogram nhitsouth not found" << std::endl;
    }

    if (pid) {
        m_canvas->cd(4);
        pid->GetXaxis()->SetTitle("Packet ID");
        pid->Draw();
    } else {
        std::cerr << "Histogram pid not found" << std::endl;
    }

    if (adc) {
        m_canvas->cd(5);
        adc->Draw();
    } else {
        std::cerr << "Histogram adc not found" << std::endl;
    }

    if (bco) {
        m_canvas->cd(6);
        bco->GetXaxis()->SetTitle("Crossing");
        bco->Draw();
    } else {
        std::cerr << "Histogram bco not found" << std::endl;
    }

    if (bcofull) {
        m_canvas->cd(7);
        bcofull->Draw();
    } else {
        std::cerr << "Histogram bcofull not found" << std::endl;
    }

    // Add text to the last pad
    TText PrintRun;
    PrintRun.SetTextFont(62);
    PrintRun.SetTextSize(0.06);
    PrintRun.SetNDC();
    PrintRun.SetTextAlign(23);
    std::ostringstream runnostream1;
    std::string runstring1;
    runnostream1 << "INTT Summary Info Run <RunNumber>, build <BuildNumber>";
    runstring1 = runnostream1.str();
    m_canvas->cd(8);
    PrintRun.DrawText(0.5, 0.5, runstring1.c_str());

    // Update and save the canvas
    m_canvas->Update();
    //m_canvas->SaveAs("INTT_Summary_Plots.pdf");

    std::cout << "Plotting completed" << std::endl;
}



## Errors 

In [13]:
%%cpp

// Call the function to execute the plotting
INTTRawHitSummaryDrawer_v2();

Plotting completed


## Pulls

In [14]:
%%cpp -d
#include <TFile.h>
#include <TCanvas.h>
#include <TH1.h>
#include <TText.h>
#include <TString.h>
#include <TPad.h>
#include <TStyle.h>
#include <iostream>
#include <sstream>

TFile* file = nullptr;

void OpenROOTFile() {
    // Open the ROOT file
    file = TFile::Open("prdf_reconstruction53877_qa.root");
    if (!file || file->IsZombie()) {
        std::cerr << "Error opening root file" << std::endl;
        return;
    }
}

void INTTRawHitChip();
void INTTRawHitSummaryDrawer_v2();
//void INTTChipDrawer();

void INTTRawHitChip() {
    if (!file) {
        std::cerr << "File not opened. Call OpenROOTFile() first." << std::endl;
        return;
    }

    // Create a canvas
    TCanvas* canvas = new TCanvas("canvas", "INTT Plots 0", 1000, 800);
    canvas->Divide(4, 4);

    // Get and draw the histograms
    for (int lad = 0; lad < 14; lad++) {
        std::ostringstream hist_name;
        hist_name << "h_InttRawHitQA_intt" << 6 << "_" << lad;
        TH1* h_nhits = dynamic_cast<TH1*>(file->Get(hist_name.str().c_str()));
        canvas->cd(lad + 1);
        gPad->SetRightMargin(0.13);
        gPad->SetLeftMargin(0.13);
        if (h_nhits) {
            h_nhits->SetTitle(Form("FEE %d", lad));
            h_nhits->Draw("colz");
        } else {
            std::cerr << "Histogram " << hist_name.str() << " not found" << std::endl;
        }
    }

    // Add text to the last pad
    TText PrintRun;
    PrintRun.SetTextFont(62);
    PrintRun.SetTextSize(0.06);
    PrintRun.SetNDC();          // set to normalized coordinates
    PrintRun.SetTextAlign(23);  // center/top alignment
    std::string runstring1 = "";
    canvas->cd(15);
    PrintRun.DrawText(0.5, 0.5, runstring1.c_str());

    // Update and save the canvas
    canvas->Update();
    //canvas->SaveAs("INTT_Plots.pdf");

    std::cout << "Plotting completed" << std::endl;
}

void INTTRawHitSummaryDrawer_v2() {
    if (!file) {
        std::cerr << "File not opened. Call OpenROOTFile() first." << std::endl;
        return;
    }

    gStyle->SetPalette(kRainBow);

    TCanvas* m_canvas = new TCanvas("m_canvas", "INTT Hits Summary", 800, 600);
    m_canvas->Divide(2, 4);

    // Get histograms
    TH1* nhit = dynamic_cast<TH1*>(file->Get("h_InttRawHitQA_nhit"));
    TH1* nhitnorth = dynamic_cast<TH1*>(file->Get("h_InttRawHitQA_nhit_north"));
    TH1* nhitsouth = dynamic_cast<TH1*>(file->Get("h_InttRawHitQA_nhit_south"));
    TH1* pid = dynamic_cast<TH1*>(file->Get("h_InttRawHitQA_pid"));
    TH1* adc = dynamic_cast<TH1*>(file->Get("h_InttRawHitQA_adc"));
    TH1* bco = dynamic_cast<TH1*>(file->Get("h_InttRawHitQA_bco"));
    TH1* bcofull = dynamic_cast<TH1*>(file->Get("h_InttRawHitQA_bco_full"));

    // Draw histograms to canvas
    if (nhit) {
        m_canvas->cd(1);
        nhit->GetXaxis()->SetTitle("N_{hit}");
        nhit->Draw();
    } else {
        std::cerr << "Histogram nhit not found" << std::endl;
    }

    if (nhitnorth) {
        m_canvas->cd(2);
        nhitnorth->GetYaxis()->SetTitle("N_{hit}");
        nhitnorth->SetTitle("INTT Hits North");
        nhitnorth->Draw();
    } else {
        std::cerr << "Histogram nhitnorth not found" << std::endl;
    }

    if (nhitsouth) {
        m_canvas->cd(3);
        nhitsouth->GetYaxis()->SetTitle("N_{hit}");
        nhitsouth->SetTitle("INTT Hits South");
        nhitsouth->Draw();
    } else {
        std::cerr << "Histogram nhitsouth not found" << std::endl;
    }

    if (pid) {
        m_canvas->cd(4);
        pid->GetXaxis()->SetTitle("Packet ID");
        pid->Draw();
    } else {
        std::cerr << "Histogram pid not found" << std::endl;
    }

    if (adc) {
        m_canvas->cd(5);
        adc->Draw();
    } else {
        std::cerr << "Histogram adc not found" << std::endl;
    }

    if (bco) {
        m_canvas->cd(6);
        bco->GetXaxis()->SetTitle("Crossing");
        bco->Draw();
    } else {
        std::cerr << "Histogram bco not found" << std::endl;
    }

    if (bcofull) {
        m_canvas->cd(7);
        bcofull->Draw();
    } else {
        std::cerr << "Histogram bcofull not found" << std::endl;
    }

    // Add text to the last pad
    TText PrintRun;
    PrintRun.SetTextFont(62);
    PrintRun.SetTextSize(0.06);
    PrintRun.SetNDC();
    PrintRun.SetTextAlign(23);
    std::ostringstream runnostream1;
    std::string runstring1;
    runnostream1 << "INTT Summary Info Run <RunNumber>, build <BuildNumber>";
    runstring1 = runnostream1.str();
    m_canvas->cd(8);
    PrintRun.DrawText(0.5, 0.5, runstring1.c_str());

    // Update and save the canvas
    m_canvas->Update();
    //m_canvas->SaveAs("INTT_Summary_Plots.pdf");

    std::cout << "Plotting completed" << std::endl;
}



input_line_96:26:6: error: redefinition of 'INTTRawHitChip'
void INTTRawHitChip() {
     ^
input_line_59:2:6: note: previous definition is here
void INTTRawHitChip() {
     ^
input_line_96:69:6: error: redefinition of 'INTTRawHitSummaryDrawer_v2'
void INTTRawHitSummaryDrawer_v2() {
     ^
input_line_92:2:6: note: previous definition is here
void INTTRawHitSummaryDrawer_v2() {
     ^


In [15]:
%%cpp
INTTRawHitChip();

Plotting completed


Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas


# $z$ residuals, errors and pulls

## Residuals

In [16]:
%%cpp

// Call the function to execute the plotting
INTTRawHitSummaryDrawer_v2();

Plotting completed


Warning in <TCanvas::Constructor>: Deleting canvas with same name: m_canvas


In [17]:
%%cpp

INTTChipDrawer();

input_line_100:2:3: error: use of undeclared identifier 'INTTChipDrawer'
 (INTTChipDrawer())
  ^
Error in <HandleInterpreterException>: Error evaluating expression (INTTChipDrawer())
Execution of your code was aborted.


In [18]:
%%cpp -d

TFile* file1 = nullptr;

void OpenROOTFile1() {
    // Open the ROOT file
    file1 = TFile::Open("prdf_reconstruction53877_qa.root");
    if (!file1 || file1->IsZombie()) {
        std::cerr << "Error opening root file" << std::endl;
        return;
    }
}

void INTTChipDrawer1() {
    if (!file1) {
        std::cerr << "File not opened. Call OpenROOTFile() first." << std::endl;
        return;
    }

    TCanvas* m_canvas_chip = new TCanvas("m_canvas_chip", "INTT Plots 0", 1000, 800);
    m_canvas_chip->Divide(2, 2);
  
    TPad* Pad[4];
    Pad[0] = new TPad("mypad_0_0", "put",  0.05, 0.52, 0.45, 0.97, 0);
    Pad[1] = new TPad("mypad_0_1", "a",    0.5,  0.52, 0.95, 0.97, 0);
    Pad[2] = new TPad("mypad_0_2", "name", 0.05, 0.02, 0.45, 0.47, 0);
    Pad[3] = new TPad("mypad_0_3", "here", 0.5,  0.02, 0.95, 0.47, 0);
  
    Pad[0]->Draw();
    Pad[1]->Draw();
    Pad[2]->Draw();
    Pad[3]->Draw();
  
    // this one is used to plot the run number on the canvas
    TPad* transparent = new TPad("transparent_0", "this does not show", 0, 0, 1, 1);
    transparent->SetFillStyle(4000);
    transparent->Draw();
    
    // Get histograms
    TH1F* h_occupancy = dynamic_cast<TH1F*>(file1->Get("h_InttClusterQA_sensorOccupancy"));
    TH1F* h_clusSize = dynamic_cast<TH1F*>(file1->Get("h_InttClusterQA_clusterSize"));
    
    // Draw histograms to canvas
    Pad[0]->cd();
    if (h_occupancy) {
        h_occupancy->SetTitle("INTT Chip Occupancy");
        h_occupancy->SetXTitle("Chip Occupancy [%]");
        h_occupancy->SetYTitle("Normalized Entries");
        h_occupancy->Scale(1.0 / h_occupancy->Integral());
        h_occupancy->DrawCopy();
        gPad->SetLogy();
        gPad->SetRightMargin(0.15);
    } else {
        std::cerr << "Histogram h_occupancy not found" << std::endl;
        return;
    }

    Pad[1]->cd();
    if (h_clusSize) {
        h_clusSize->SetTitle("INTT Cluster Size");
        h_clusSize->SetXTitle("Cluster Size");
        h_clusSize->SetYTitle("Normalized Entries");
        h_clusSize->Scale(1.0 / h_clusSize->Integral());
        h_clusSize->DrawCopy();
        gPad->SetRightMargin(0.15);
    } else {
        std::cerr << "Histogram h_clusSize not found" << std::endl;
        return;
    }

    TText PrintRun;
    PrintRun.SetTextFont(62);
    PrintRun.SetTextSize(0.04);
    PrintRun.SetNDC();  // set to normalized coordinates
    PrintRun.SetTextAlign(23); // center/top alignment
    std::ostringstream runnostream1;
    std::string runstring1;
    runnostream1 << "INTTQA_intt Info Run <RunNumber>, build <BuildNumber>";
    runstring1 = runnostream1.str();
    transparent->cd();
    PrintRun.DrawText(0.5, 1.0, runstring1.c_str());

    m_canvas_chip->Update();
 
    std::cout << "DrawChipInfo completed" << std::endl;
}

int main() {
    OpenROOTFile1();
    INTTChipDrawer1();
    return 0;
}

In [19]:
%%cpp

// Call the function to execute the plotting
main();

DrawChipInfo completed
